# Validación de modelos

Hemos construido un modelo. Pero, ¿qué tan bueno es?

En esta lección, aprenderemos a usar la validación del modelo para medir la calidad de nuestro modelo. Medir la calidad del modelo es la clave para mejorar nuestros modelos de manera iterativa.

## ¿Qué es la validación de modelos?

Siempre vamos a querer evaluar casi todos los modelos que construyamos. En la mayoría de las aplicaciones (aunque no en todas), la medida relevante de la calidad del modelo es la precisión predictiva. En otras palabras, ¿las predicciones del modelo estarán cerca de lo que realmente sucede?

Muchas personas cometen un gran error al medir la precisión predictiva. Hacen predicciones con sus datos de entrenamiento y comparan esas predicciones con los valores objetivo en los datos de entrenamiento. Veremos el problema con este enfoque y cómo solucionarlo en un momento, pero primero pensemos en cómo lo haríamos.

Primero necesitaríamos resumir la calidad del modelo de una manera comprensible. Si comparamos los valores predichos y reales de 10,000 casas, probablemente encontraremos una mezcla de buenas y malas predicciones. Revisar una lista de 10,000 valores predichos y reales sería inútil. Necesitamos resumir esto en una sola métrica.

Existen muchas métricas para resumir la calidad del modelo, pero comenzaremos con una llamada Error Absoluto Medio (Mean Absolute Error, también conocido como MAE). Desglosaremos esta métrica comenzando con la última palabra, error.

El error de predicción para cada casa es:

$$
error = real − predicción
$$

Entonces, si una casa costó 150,000 y predijiste que costaría 100,000, el error es 50,000.

Con la métrica MAE, tomamos el valor absoluto de cada error. Esto convierte cada error en un número positivo. Luego tomamos el promedio de esos errores absolutos. Esta es nuestra medida de la calidad del modelo. En lenguaje sencillo, se puede decir:

> En promedio, nuestras predicciones están equivocadas por aproximadamente X.

Para calcular el MAE, primero necesitamos un modelo.

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor

# Cargar datos
ruta = './data/melb_data.csv'
melbourne_data = pd.read_csv(ruta) 

# Quitar nulos
melbourne_data_filtrada = melbourne_data.dropna(axis=0)

# Elegir target
y = melbourne_data_filtrada.Price

# Elegir features
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude']

# Crear X
X = melbourne_data_filtrada[melbourne_features]


# Definir modelo
modelo = DecisionTreeRegressor()

# Ajustar modelo
modelo.fit(X, y)

Ya tenemos nuestro modelo y ahora podemos calcular el MAE. Para esto utilizaremos también `sklearn`.

In [ ]:
from sklearn.metrics import mean_absolute_error

predicciones = modelo.predict(X)
mean_absolute_error(y, predicciones)

## El Problema con los puntajes "dentro de la muestra"

La medida que acabamos de calcular puede llamarse un puntaje "dentro de la muestra". Usamos una sola "muestra" de casas tanto para construir el modelo como para evaluarlo. Esto está mal. Veamos por qué.

Imaginemos que en el mercado inmobiliario el color de la puerta no está relacionado con el precio de la casa.

Sin embargo, en la muestra de datos que usamos para construir el modelo, todas las casas con puertas verdes eran muy caras. El trabajo del modelo es encontrar patrones que predigan los precios de las casas, así que verá este patrón y siempre predecirá precios altos para las casas con puertas verdes.

Dado que este patrón se derivó de los datos de entrenamiento, el modelo parecerá preciso en los datos de entrenamiento.

Pero si este patrón no se mantiene cuando el modelo ve nuevos datos, el modelo sería muy impreciso cuando se use en la práctica.

Dado que el valor práctico de los modelos proviene de hacer predicciones sobre nuevos datos, medimos el rendimiento en datos que no se usaron para construir el modelo. La forma más sencilla de hacer esto es excluir algunos datos del proceso de construcción del modelo y luego usarlos para probar la precisión del modelo en datos que no ha visto antes. Estos datos se llaman datos de validación.

## Partiendo los datos
Vamos a dividir nuestros datos en 2 subconjuntos: set de entrenamiento y set de pruebas.

`scikit-learn` tiene una función llamada `train_test_split` para dividir los datos en estas dos partes. Usaremos algunos de esos datos como datos de entrenamiento para ajustar el modelo, y usaremos los otros datos como datos de validación para calcular el error absoluto medio (`mean_absolute_error`).

In [ ]:
from sklearn.model_selection import train_test_split

Dividimos los datos en datos de entrenamiento y de validación, tanto para las características como para el objetivo

Esta división se basa en un generador de números aleatorios. Por lo tanto, podemos darle un `random_state` para que siempre que corramos nuestro código, tengamamos exactamente los mismos sets de entrenamiento y de pruebas.

La función `train_test_split` recibe 2 argumentos y 1 opcional. El primer argumento es el dataframe que contiene los features, y el segundo argumento contiene los precios (targets). 

La función nos regresará 4 conjuntos de datos. 2 para entrenar y 2 para probar

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)

In [ ]:
train_X.head()

In [ ]:
train_y.head()

In [ ]:
val_X.head()

In [ ]:
val_y.head()

Ahora definamos nuestro modelo nuevamento y ajustémoslo

In [ ]:
modelo = DecisionTreeRegressor()

# Ajustar únicamente con datos de entrenamiento
modelo.fit(train_X, train_y)

# obtener predicciones
val_predictions = modelo.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

**¡¿Ves la diferencia?!**

Cuando validamos nuestro modelo con los mismos datos con los que lo entrenamos, obtubimos un error de alrededor 500 dólares, pero ahora tenemos un error de 263,515 dólares!

---

Ésta es la diferencia entre un modelo que es casi exactamente correcto y uno que es inutilizable para la mayoría de los propósitos prácticos. Como punto de referencia, el valor promedio de una casa en los datos de validación es de 1.1 millones de dólares. Por lo tanto, el error en los nuevos datos es aproximadamente una cuarta parte del valor promedio de la casa.

Hay muchas maneras de mejorar este modelo, como experimentar para encontrar mejores características o diferentes tipos de modelos.